# Recommending Items Recently Bought

In [ ]:
!pip install modin
import modin.pandas as pd
import ray
ray.init()
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.head(1)

In [ ]:
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

In [ ]:
# customers.head()

In [ ]:
# articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')

In [ ]:
# articles.head(2)

In [ ]:
# articles[articles.columns].nunique()

In [ ]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', dtype={'article_id': str})
# set dtype or pandas will drop the leading '0' and convert to int

# transactions_train = pd.read_csv('../input/recomen-split/train0.csv', dtype={'article_id': str})


In [ ]:
transactions_train['t_dat'].unique()

In [ ]:
transactions_train.shape

In [ ]:
transactions_train['t_dat'] = pd.to_datetime(transactions_train['t_dat'])
part = transactions_train[transactions_train['t_dat'] >= pd.to_datetime('2019-07-01')]

In [ ]:
transactions_trainR = part[::-1]

In [ ]:
part = transactions_train[transactions_train['t_dat'] < pd.to_datetime('2019-07-01')]
partR = part[::-1]

In [ ]:
transactions_trainR = pd.concat([transactions_trainR, partR])
transactions_trainR

In [ ]:
# transactions_trainR.to_csv('transactions_trainR.csv', index = False)


In [ ]:
t_dat_unique = transactions_trainR['t_dat'].unique()
# len(t_dat_unique) # 734
t_dat_number = np.arange(len(t_dat_unique))

In [ ]:
len(t_dat_unique)

In [ ]:
t_dat_unique

In [ ]:
temp_submission = sample_submission.copy()
# temp_submission.index = temp_submission['customer_id']
temp_submission.head(3)

In [ ]:
transactions_trainR['article_id'] = transactions_trainR['article_id'] + ' '
transactions_trainR

In [ ]:
g1 = transactions_trainR.groupby('customer_id')['article_id'].sum()#apply(lambda x: x.sum())

df1 = pd.DataFrame(
    data=np.array(g1.index),
#     index=g1.index,
    columns=['customer_id']
             )

df2 = pd.DataFrame(
    data=np.array(g1),
#     index=g1.index,
    columns=['article_id']
             )
df_add = df1.join(df2)

In [ ]:
new_temp_submission = temp_submission.set_index('customer_id')
mini_temp = df_add.set_index('customer_id')
new_temp_submission2 = new_temp_submission.join(mini_temp, how='left')
new_temp_submission2 = new_temp_submission2.reset_index()
# new_temp_submission2# = new_temp_submission2.rename(columns={'index': 'customer_id'})
new_temp_submission2['new_prediction'] = new_temp_submission2['article_id'] + new_temp_submission2['prediction']
temp_submission['prediction'] = new_temp_submission2['new_prediction']
temp_submission

In [ ]:
temp_submission.isnull().any()

In [ ]:
temp_submission['prediction'][temp_submission['prediction'].isnull()] = sample_submission['prediction']

In [ ]:
temp_submission.isnull().any()

In [ ]:
sample_submission['prediction'] = temp_submission['prediction'].str[:131]

In [ ]:
sample_submission.isnull().any()

In [ ]:
sample_submission.to_csv('submission.csv', index = False)

In [ ]:
sample_submission